In [1]:
import sys
import os
import pickle
import logging
import numba
import matplotlib
import matplotlib.pyplot as plt
logging.getLogger('matplotlib.font_manager').disabled = True
numba_logger = logging.getLogger('numba')
numba_logger.setLevel(logging.WARNING)

matplotlib_logger = logging.getLogger('matplotlib')
matplotlib_logger.setLevel(logging.WARNING)

# Add the src directory to sys.path
src_path = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
print(src_path)
sys.path.append(src_path)

from src.IndependentSteps import Pycromanager2NativeDataType, FFF2NativeDataType

from src.SequentialSteps import BIGFISH_SpotDetection, SimpleCellposeSegmentaion, IlluminationCorrection, \
                                Calculate_BIGFISH_Threshold, CellProperties, Automatic_BIGFISH_Threshold

from src.FinalizationSteps import Save_Outputs, Save_Images, Save_Parameters, Save_Masks, return_to_NAS, remove_local_data_but_keep_h5, \
                                     remove_local_data

from src.Parameters import Parameters, Experiment, Settings, ScopeClass, DataContainer

from src.Displays import Display

from src.GUI import GUI, StepGUI

from src.Pipeline import Pipeline

c:\Users\formanj\GitHub\AngelFISH


INFO:numexpr.utils:Note: NumExpr detected 32 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
INFO:numexpr.utils:NumExpr defaulting to 16 threads.
DEBUG:h5py._conv:Creating converter from 7 to 5
DEBUG:h5py._conv:Creating converter from 5 to 7
DEBUG:h5py._conv:Creating converter from 7 to 5
DEBUG:h5py._conv:Creating converter from 5 to 7


In [2]:
def runner(idl, ip):
    # Initialize Parameters
    scope = ScopeClass()
    scope.voxel_size_yx = 160
    scope.spot_yx = 300
    scope.spot_z = 500 
    data = DataContainer()
    data.clear_after_error = False
    settings = Settings(name='ER_Dec0324_DUSP1', ) 
    experiment = Experiment()
    experiment.independent_params = ip
    experiment.initial_data_location = idl
    # experiment.snr_threshold = 4 # were going to do this post processing

    settings.load_in_mask = False
    experiment.FISHChannel = 0
    experiment.nucChannel = 2
    experiment.cytoChannel = 1
    experiment.voxel_size_z = 500

    settings.cellpose_min_size = 100
    settings.cellpose_diameter = [180, 90]
    settings.cellpose_pretrained_model = ["GAPDH_cyto", 'DAPI_nuclei'] 

    settings.bigfish_threshold = 'mean'
    settings.verbose = True

    FFF2NativeDataType()
    Automatic_BIGFISH_Threshold()
    SimpleCellposeSegmentaion()
    BIGFISH_SpotDetection()
    CellProperties() # this is probably not need but oh well
    Save_Masks()
    Save_Parameters()
    Save_Outputs()
    return_to_NAS()
    remove_local_data()

    Parameters.validate()

    pipeline = Pipeline()

    pipeline.run_on_cluster('/home/formanj/Github/AngelFISH/cluster', name=settings.name) # give this a slightluy better name the date is already saved 

    pipeline.clear_pipeline() # I dont think this is nesscary but its probablily good to do
    

In [3]:
# List of directories to process
initial_data_location=(
        'smFISH_images/Eric_smFISH_images/20220225/DUSP1_Dex_0min_20220224' ,
        'smFISH_images/Eric_smFISH_images/20220225/DUSP1_Dex_10min_20220224' ,
        'smFISH_images/Eric_smFISH_images/20220225/DUSP1_Dex_20min_20220224' ,
        'smFISH_images/Eric_smFISH_images/20220225/DUSP1_Dex_30min_20220224' ,
        'smFISH_images/Eric_smFISH_images/20220303/DUSP1_Dex_40min_20220224' ,
        'smFISH_images/Eric_smFISH_images/20220303/DUSP1_Dex_50min_20220224' ,
        'smFISH_images/Eric_smFISH_images/20220304/DUSP1_Dex_60min_20220224' ,
        'smFISH_images/Eric_smFISH_images/20220304/DUSP1_Dex_75min_20220224' ,
        'smFISH_images/Eric_smFISH_images/20220305/DUSP1_Dex_90min_20220224' ,
        'smFISH_images/Eric_smFISH_images/20220305/DUSP1_Dex_120min_20220224' ,
        'smFISH_images/Eric_smFISH_images/20220305/DUSP1_Dex_150min_20220224' ,
        'smFISH_images/Eric_smFISH_images/20220305/DUSP1_Dex_180min_20220224' )
# Parameters
Condition = 'DUSP1_timesweep'      # Experimental condition. The options are:  'GR_timesweep' , 'DUSP1_timesweep' , 'DUSP1_TPL'
Replica = 'D'                   # Replica name. 
time_list  = [0, 10, 20, 30, 40, 50, 60, 75, 90, 120, 150, 180]        # Time of image acquisition
DexConc_list = [0, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100]        # Dex concentration

# Creating the list of dictionaries
DUSP1_TS_R1 = [
    {
        "condition": Condition,
        "replica": Replica,
        "time": time,
        "Dex_Conc": DexConc
    }
    for time, DexConc in zip(time_list, DexConc_list)
]
# Output the result
print(DUSP1_TS_R1)

[{'condition': 'DUSP1_timesweep', 'replica': 'D', 'time': 0, 'Dex_Conc': 0}, {'condition': 'DUSP1_timesweep', 'replica': 'D', 'time': 10, 'Dex_Conc': 100}, {'condition': 'DUSP1_timesweep', 'replica': 'D', 'time': 20, 'Dex_Conc': 100}, {'condition': 'DUSP1_timesweep', 'replica': 'D', 'time': 30, 'Dex_Conc': 100}, {'condition': 'DUSP1_timesweep', 'replica': 'D', 'time': 40, 'Dex_Conc': 100}, {'condition': 'DUSP1_timesweep', 'replica': 'D', 'time': 50, 'Dex_Conc': 100}, {'condition': 'DUSP1_timesweep', 'replica': 'D', 'time': 60, 'Dex_Conc': 100}, {'condition': 'DUSP1_timesweep', 'replica': 'D', 'time': 75, 'Dex_Conc': 100}, {'condition': 'DUSP1_timesweep', 'replica': 'D', 'time': 90, 'Dex_Conc': 100}, {'condition': 'DUSP1_timesweep', 'replica': 'D', 'time': 120, 'Dex_Conc': 100}, {'condition': 'DUSP1_timesweep', 'replica': 'D', 'time': 150, 'Dex_Conc': 100}, {'condition': 'DUSP1_timesweep', 'replica': 'D', 'time': 180, 'Dex_Conc': 100}]


In [4]:
runner(initial_data_location, DUSP1_TS_R1)

DEBUG:paramiko.transport:starting thread (client mode): 0x5734e810
DEBUG:paramiko.transport:Local version/idstring: SSH-2.0-paramiko_3.5.0
DEBUG:paramiko.transport:Remote version/idstring: SSH-2.0-OpenSSH_8.0
INFO:paramiko.transport:Connected (version 2.0, client OpenSSH_8.0)
DEBUG:paramiko.transport:=== Key exchange possibilities ===
DEBUG:paramiko.transport:kex algos: curve25519-sha256, curve25519-sha256@libssh.org, ecdh-sha2-nistp256, ecdh-sha2-nistp384, ecdh-sha2-nistp521, diffie-hellman-group-exchange-sha256, diffie-hellman-group14-sha256, diffie-hellman-group16-sha512, diffie-hellman-group18-sha512, diffie-hellman-group-exchange-sha1, diffie-hellman-group14-sha1
DEBUG:paramiko.transport:server key: rsa-sha2-512, rsa-sha2-256, ssh-rsa, ecdsa-sha2-nistp256, ssh-ed25519
DEBUG:paramiko.transport:client encrypt: aes256-gcm@openssh.com, chacha20-poly1305@openssh.com, aes256-ctr, aes256-cbc, aes128-gcm@openssh.com, aes128-ctr, aes128-cbc
DEBUG:paramiko.transport:server encrypt: aes256-g

None


DEBUG:paramiko.transport:Secsh channel 0 opened.
DEBUG:paramiko.transport:[chan 0] Sesch channel 0 request ok
INFO:paramiko.transport.sftp:[chan 0] Opened sftp connection (server version 3)
DEBUG:paramiko.transport.sftp:[chan 0] open(b'/home/formanj/Github/AngelFISH/cluster/ER_Dec0324_DUSP1.txt', 'wb')
DEBUG:paramiko.transport.sftp:[chan 0] open(b'/home/formanj/Github/AngelFISH/cluster/ER_Dec0324_DUSP1.txt', 'wb') -> 00000000
DEBUG:paramiko.transport.sftp:[chan 0] close(00000000)
DEBUG:paramiko.transport.sftp:[chan 0] stat(b'/home/formanj/Github/AngelFISH/cluster/ER_Dec0324_DUSP1.txt')
INFO:paramiko.transport.sftp:[chan 0] sftp session closed.
DEBUG:paramiko.transport:[chan 0] EOF sent (0)
DEBUG:paramiko.transport:[chan 1] Max packet in: 32768 bytes
DEBUG:paramiko.transport:[chan 0] EOF received (0)
DEBUG:paramiko.transport:[chan 1] Max packet out: 32768 bytes
DEBUG:paramiko.transport:Secsh channel 1 opened.
DEBUG:paramiko.transport:[chan 1] Sesch channel 1 request ok
DEBUG:paramiko.tr

Submitted batch job 22679




In [ ]:
# List of directories to process
initial_data_location=(
        'smFISH_images/Eric_smFISH_images/20220420/DUSP1_100nM_Dex_R3_20220419_0min' ,
        'smFISH_images/Eric_smFISH_images/20220420/DUSP1_100nM_Dex_R3_20220419_10min' ,
        'smFISH_images/Eric_smFISH_images/20220420/DUSP1_100nM_Dex_R3_20220419_20min' ,
        'smFISH_images/Eric_smFISH_images/20220420/DUSP1_100nM_Dex_R3_20220419_30min' ,
        'smFISH_images/Eric_smFISH_images/20220421/DUSP1_100nM_Dex_R3_20220419_40min' ,
        'smFISH_images/Eric_smFISH_images/20220421/DUSP1_100nM_Dex_R3_20220419_50min' ,
        'smFISH_images/Eric_smFISH_images/20220421/DUSP1_100nM_Dex_R3_20220419_60min' ,
        'smFISH_images/Eric_smFISH_images/20220421/DUSP1_100nM_Dex_R3_20220419_75min' ,
        'smFISH_images/Eric_smFISH_images/20220421/DUSP1_100nM_Dex_R3_20220419_90min' ,
        'smFISH_images/Eric_smFISH_images/20220421/DUSP1_100nM_Dex_R3_20220419_120min' ,
        'smFISH_images/Eric_smFISH_images/20220425/DUSP1_100nM_Dex_R3_20220419_150min' ,
        'smFISH_images/Eric_smFISH_images/20220425/DUSP1_100nM_Dex_R3_20220419_180min' )
# Parameters
Condition = 'DUSP1_timesweep'      # Experimental condition. The options are:  'GR_timesweep' , 'DUSP1_timesweep' , 'DUSP1_TPL'
Replica = 'E'                   # Replica name. 
time_list  = [0, 10, 20, 30, 40, 50, 60, 75, 90, 120, 150, 180]        # Time of image acquisition
DexConc_list = [0, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100]        # Dex concentration

# Creating the list of dictionaries
DUSP1_TS_R2 = [
    {
        "condition": Condition,
        "replica": Replica,
        "time": time,
        "Dex_Conc": DexConc
    }
    for time, DexConc in zip(time_list, DexConc_list)
]
# Output the result
print(DUSP1_TS_R2)

In [ ]:
runner(initial_data_location, DUSP1_TS_R2)

In [ ]:
# List of directories to process
initial_data_location=(
        'smFISH_images/Eric_smFISH_images/20220602/DUSP1_Dex_100nM_0min_NoSpin_052722' ,
        'smFISH_images/Eric_smFISH_images/20220602/DUSP1_Dex_100nM_10min_NoSpin_052722' ,
        'smFISH_images/Eric_smFISH_images/20220602/DUSP1_Dex_100nM_20min_NoSpin_052722' ,
        'smFISH_images/Eric_smFISH_images/20220602/DUSP1_Dex_100nM_30min_NoSpin_052722' ,
        'smFISH_images/Eric_smFISH_images/20220602/DUSP1_Dex_100nM_40min_NoSpin_052722' ,
        'smFISH_images/Eric_smFISH_images/20220602/DUSP1_Dex_100nM_50min_NoSpin_052722' ,
        'smFISH_images/Eric_smFISH_images/20220606/DUSP1_Dex_100nM_60min_NoSpin_052722' ,
        'smFISH_images/Eric_smFISH_images/20220606/DUSP1_Dex_100nM_75min_NoSpin_052722' ,
        'smFISH_images/Eric_smFISH_images/20220606/DUSP1_Dex_100nM_90min_NoSpin_052722' ,
        'smFISH_images/Eric_smFISH_images/20220606/DUSP1_Dex_100nM_120min_NoSpin_052722' ,
        'smFISH_images/Eric_smFISH_images/20220606/DUSP1_Dex_100nM_150min_NoSpin_052722' ,
        'smFISH_images/Eric_smFISH_images/20220606/DUSP1_Dex_100nM_180min_NoSpin_052722' )
# Parameters
Condition = 'DUSP1_timesweep'      # Experimental condition. The options are:  'GR_timesweep' , 'DUSP1_timesweep' , 'DUSP1_TPL'
Replica = 'F'                   # Replica name. 
time_list  = [0, 10, 20, 30, 40, 50, 60, 75, 90, 120, 150, 180]        # Time of image acquisition
DexConc_list = [0, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100]        # Dex concentration

# Creating the list of dictionaries
DUSP1_TS_R3 = [
    {
        "condition": Condition,
        "replica": Replica,
        "time": time,
        "Dex_Conc": DexConc
    }
    for time, DexConc in zip(time_list, DexConc_list)
]
# Output the result
print(DUSP1_TS_R3)

In [ ]:
runner(initial_data_location, DUSP1_TS_R3)


# Concentration Sweep

In [ ]:
# List of directories to process
initial_data_location=(
        'smFISH_images/Eric_smFISH_images/20220628/DUSP1_conc_sweep_0min_060322' ,
        'smFISH_images/Eric_smFISH_images/20220707/DUSP1_conc_sweep_1pM_75min_060322'  ,
        'smFISH_images/Eric_smFISH_images/20220707/DUSP1_conc_sweep_10pM_75min_060322' ,
        'smFISH_images/Eric_smFISH_images/20220707/DUSP1_conc_sweep_100pM_75min_060322' ,
        'smFISH_images/Eric_smFISH_images/20220705/DUSP1_conc_sweep_1nM_75min_060322' ,
        'smFISH_images/Eric_smFISH_images/20220705/DUSP1_conc_sweep_10nM_75min_060322' ,
        'smFISH_images/Eric_smFISH_images/20220705/DUSP1_conc_sweep_100nM_75min_060322' ,
        'smFISH_images/Eric_smFISH_images/20220705/DUSP1_conc_sweep_1uM_75min_060322' ,
        'smFISH_images/Eric_smFISH_images/20220628/DUSP1_conc_sweep_10uM_75min_060322')
# Parameters
Condition = 'DUSP1_timesweep'      # Experimental condition. The options are:  'GR_timesweep' , 'DUSP1_timesweep' , 'DUSP1_TPL'
Replica = 'G'                   # Replica name. 
time_list  = [0, 75, 75, 75, 75, 75, 75, 75, 75]        # Time of image acquisition
DexConc_list = [0, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]        # Dex concentration

# Creating the list of dictionaries
DUSP1_CS_R1 = [
    {
        "condition": Condition,
        "replica": Replica,
        "time": time,
        "Dex_Conc": DexConc
    }
    for time, DexConc in zip(time_list, DexConc_list)
]
# Output the result
print(DUSP1_CS_R1)

In [ ]:
runner(initial_data_location, DUSP1_CS_R1)


In [ ]:
# List of directories to process
initial_data_location=(
        'smFISH_images/Eric_smFISH_images/20220718/DUSP1_conc_sweep_R2_0min_071422' ,
        'smFISH_images/Eric_smFISH_images/20220721/DUSP1_conc_sweep_R2_1pM_75min_071422' ,   
        'smFISH_images/Eric_smFISH_images/20220721/DUSP1_conc_sweep_R2_10pM_75min_071422' ,    
        'smFISH_images/Eric_smFISH_images/20220721/DUSP1_conc_sweep_R2_100pM_75min_071422' ,    
        'smFISH_images/Eric_smFISH_images/20220718/DUSP1_conc_sweep_R2_1nM_75min_071422' ,    
        'smFISH_images/Eric_smFISH_images/20220718/DUSP1_conc_sweep_R2_10nM_75min_071422' ,    
        'smFISH_images/Eric_smFISH_images/20220718/DUSP1_conc_sweep_R2_1uM_75min_071422' ,
        'smFISH_images/Eric_smFISH_images/20220718/DUSP1_conc_sweep_R2_10uM_75min_071422' )
# Parameters
Condition = 'DUSP1_timesweep'      # Experimental condition. The options are:  'GR_timesweep' , 'DUSP1_timesweep' 
Replica = 'H'                   # Replica name. 
time_list  = [0, 75, 75, 75, 75, 75, 75, 75]        # Time of image acquisition
DexConc_list = [0, 0.001, 0.01, 0.1, 1, 10, 1000, 10000]        # Dex concentration

# Creating the list of dictionaries
DUSP1_CS_R2 = [
    {
        "condition": Condition,
        "replica": Replica,
        "time": time,
        "Dex_Conc": DexConc
    }
    for time, DexConc in zip(time_list, DexConc_list)
]
# Output the result
print(DUSP1_CS_R2)

In [ ]:
runner(initial_data_location, DUSP1_CS_R2)


In [ ]:
# List of directories to process
initial_data_location=(
        'smFISH_images/Eric_smFISH_images/20221003/DUSP1_Dex_Sweep_0nM_0min_Control_092022' ,
        'smFISH_images/Eric_smFISH_images/20221003/DUSP1_Dex_Sweep_1pM_75min_092022' ,
        'smFISH_images/Eric_smFISH_images/20221003/DUSP1_Dex_Sweep_10pM_75min_092022' ,
        'smFISH_images/Eric_smFISH_images/20221003/DUSP1_Dex_Sweep_100pM_75min_092022' ,
        'smFISH_images/Eric_smFISH_images/20221006/DUSP1_Dex_Sweep_1nM_75min_092022' ,
        'smFISH_images/Eric_smFISH_images/20221006/DUSP1_Dex_Sweep_10nM_75min_092022' ,
        'smFISH_images/Eric_smFISH_images/20221006/DUSP1_Dex_Sweep_100nM_75min_092022' ,
        'smFISH_images/Eric_smFISH_images/20221011/DUSP1_Dex_Sweep_1uM_75min_092022' ,
        'smFISH_images/Eric_smFISH_images/20221011/DUSP1_Dex_Sweep_10uM_75min_092022' )
# Parameters
Condition = 'DUSP1_timesweep'      # Experimental condition. The options are:  'GR_timesweep' , 'DUSP1_timesweep' 
Replica = 'I'                   # Replica name. 
time_list  = [0, 75, 75, 75, 75, 75, 75, 75, 75]        # Time of image acquisition
DexConc_list = [0, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]        # Dex concentration

# Creating the list of dictionaries
DUSP1_CS_R3 = [
    {
        "condition": Condition,
        "replica": Replica,
        "time": time,
        "Dex_Conc": DexConc
    }
    for time, DexConc in zip(time_list, DexConc_list)
]
# Output the result
print(DUSP1_CS_R3)

In [ ]:
runner(initial_data_location, DUSP1_CS_R3)


In [ ]:
# List of directories to process
initial_data_location=(
        'smFISH_images/Eric_smFISH_images/20230306/DUSP1_0nM_Dex_0min_012623' ,
        'smFISH_images/Eric_smFISH_images/20230306/DUSP1_300pM_Dex_30min_012623' ,
        'smFISH_images/Eric_smFISH_images/20230306/DUSP1_300pM_Dex_50min_012623' ,
        'smFISH_images/Eric_smFISH_images/20230306/DUSP1_300pM_Dex_75min_012623' ,
        'smFISH_images/Eric_smFISH_images/20230309/DUSP1_300pM_Dex_90min_012623' ,
        'smFISH_images/Eric_smFISH_images/20230309/DUSP1_300pM_Dex_120min_012623' ,
        'smFISH_images/Eric_smFISH_images/20230309/DUSP1_300pM_Dex_180min_012623' ,
        'smFISH_images/Eric_smFISH_images/20230309/DUSP1_1nM_Dex_30min_012623' ,
        'smFISH_images/Eric_smFISH_images/20230310/DUSP1_1nM_Dex_50min_012623' ,
        'smFISH_images/Eric_smFISH_images/20230310/DUSP1_1nM_Dex_75min_012623' ,
        'smFISH_images/Eric_smFISH_images/20230310/DUSP1_1nM_Dex_90min_012623' ,
        'smFISH_images/Eric_smFISH_images/20230310/DUSP1_1nM_Dex_120min_012623' ,
        'smFISH_images/Eric_smFISH_images/20230316/DUSP1_1nM_Dex_180min_012623' ,
        'smFISH_images/Eric_smFISH_images/20230316/DUSP1_10nM_Dex_30min_012623' ,
        'smFISH_images/Eric_smFISH_images/20230316/DUSP1_10nM_Dex_50min_012623' ,
        'smFISH_images/Eric_smFISH_images/20230316/DUSP1_10nM_Dex_75min_012623' ,
        'smFISH_images/Eric_smFISH_images/20230316/DUSP1_10nM_Dex_90min_012623' ,
        'smFISH_images/Eric_smFISH_images/20230316/DUSP1_10nM_Dex_120min_012623' ,
        'smFISH_images/Eric_smFISH_images/20230316/DUSP1_10nM_Dex_180min_012623' )
# Parameters
Condition = 'DUSP1_timesweep'      # Experimental condition. The options are:  'GR_timesweep' , 'DUSP1_timesweep' 
Replica = 'J'                   # Replica name. 
time_list  = [0, 30, 50, 75, 90, 120, 180, 30, 50, 75, 90, 120, 180, 30, 50, 75, 90, 120, 180]        # Time of image acquisition
DexConc_list = [0, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 1, 1, 1, 1, 1, 1, 10, 10, 10, 10, 10, 10]        # Dex concentration

# Creating the list of dictionaries
DUSP1_TCS_R1 = [
    {
        "condition": Condition,
        "replica": Replica,
        "time": time,
        "Dex_Conc": DexConc
    }
    for time, DexConc in zip(time_list, DexConc_list)
]
# Output the result
print(DUSP1_TCS_R1)

In [ ]:
runner(initial_data_location, DUSP1_TCS_R1)


In [ ]:
# List of directories to process
initial_data_location=(
        'smFISH_images/Eric_smFISH_images/20230413/DUSP1_DexTimeConcSweep_0min_041223' ,
        'smFISH_images/Eric_smFISH_images/20230413/DUSP1_DexTimeConcSweep_300pM_30min_041223' ,
        'smFISH_images/Eric_smFISH_images/20230413/DUSP1_DexTimeConcSweep_300pM_50min_041223' ,
        'smFISH_images/Eric_smFISH_images/20230413/DUSP1_DexTimeConcSweep_300pM_75min_041223' ,
        'smFISH_images/Eric_smFISH_images/20230413/DUSP1_DexTimeConcSweep_300pM_90min_041223' ,
        'smFISH_images/Eric_smFISH_images/20230424/DUSP1_DexTimeConcSweep_300pM_120min_041223' ,
        'smFISH_images/Eric_smFISH_images/20230424/DUSP1_DexTimeConcSweep_300pM_180min_041223' ,
        'smFISH_images/Eric_smFISH_images/20230424/DUSP1_DexTimeConcSweep_1nM_30min_041223' ,
        'smFISH_images/Eric_smFISH_images/20230424/DUSP1_DexTimeConcSweep_1nM_50min_041223' ,
        'smFISH_images/Eric_smFISH_images/20230424/DUSP1_DexTimeConcSweep_1nM_75min_041223' ,
        'smFISH_images/Eric_smFISH_images/20230424/DUSP1_DexTimeConcSweep_1nM_90min_041223' ,
        'smFISH_images/Eric_smFISH_images/20230427/DUSP1_DexTimeConcSweep_1nM_120min_041223' ,
        'smFISH_images/Eric_smFISH_images/20230427/DUSP1_DexTimeConcSweep_1nM_180min_041223' ,
        'smFISH_images/Eric_smFISH_images/20230427/DUSP1_DexTimeConcSweep_10nM_30min_041223' ,
        'smFISH_images/Eric_smFISH_images/20230427/DUSP1_DexTimeConcSweep_10nM_50min_041223' ,
        'smFISH_images/Eric_smFISH_images/20230511/DUSP1_DexTimeConcSweep_10nM_75min_041223' ,
        'smFISH_images/Eric_smFISH_images/20230511/DUSP1_DexTimeConcSweep_10nM_90min_041223' ,
        'smFISH_images/Eric_smFISH_images/20230511/DUSP1_DexTimeConcSweep_10nM_120min_041223' ,
        'smFISH_images/Eric_smFISH_images/20230511/DUSP1_DexTimeConcSweep_10nM_180min_041223' )
# Parameters
Condition = 'DUSP1_timesweep'      # Experimental condition. The options are:  'GR_timesweep' , 'DUSP1_timesweep' , 'DUSP1_TPL'
Replica = 'K'                   # Replica name. 
time_list  = [0, 30, 50, 75, 90, 120, 180, 30, 50, 75, 90, 120, 180, 30, 50, 75, 90, 120, 180]        # Time of image acquisition
DexConc_list = [0, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 1, 1, 1, 1, 1, 1, 10, 10, 10, 10, 10, 10]        # Dex concentration

# Creating the list of dictionaries
DUSP1_TCS_R2 = [
    {
        "condition": Condition,
        "replica": Replica,
        "time": time,
        "Dex_Conc": DexConc
    }
    for time, DexConc in zip(time_list, DexConc_list)
]
# Output the result
print(DUSP1_TCS_R2)

In [ ]:
runner(initial_data_location, DUSP1_TCS_R2)


In [ ]:
# List of directories to process
initial_data_location=(
        'smFISH_images/Eric_smFISH_images/20230530/DUSP1_Dex_time_conc_sweep_0min_050223_R3' ,
        'smFISH_images/Eric_smFISH_images/20230530/DUSP1_Dex_300pM_30min_050223_R3' ,
        'smFISH_images/Eric_smFISH_images/20230530/DUSP1_Dex_300pM_50min_050223_R3' ,
        'smFISH_images/Eric_smFISH_images/20230605/DUSP1_Dex_300pM_75min_050223_R3' ,
        'smFISH_images/Eric_smFISH_images/20230605/DUSP1_Dex_300pM_90min_050223_R3' ,
        'smFISH_images/Eric_smFISH_images/20230605/DUSP1_Dex_300pM_120min_050223_R3' ,
        'smFISH_images/Eric_smFISH_images/20230605/DUSP1_Dex_300pM_180min_050223_R3' ,
        'smFISH_images/Eric_smFISH_images/20230605/DUSP1_Dex_1nM_30min_050223_R3' ,
        'smFISH_images/Eric_smFISH_images/20230605/DUSP1_Dex_1nM_50min_050223_R3' ,
        'smFISH_images/Eric_smFISH_images/20230608/DUSP1_Dex_1nM_90min_050223_R3' ,
        'smFISH_images/Eric_smFISH_images/20230608/DUSP1_Dex_1nM_120min_050223_R3' ,
        'smFISH_images/Eric_smFISH_images/20230608/DUSP1_Dex_1nM_180min_050223_R3' ,
        'smFISH_images/Eric_smFISH_images/20230614/DUSP1_Dex_10nM_30min_050223_R3_redo' ,
        'smFISH_images/Eric_smFISH_images/20230608/DUSP1_Dex_10nM_50min_050223_R3' ,
        'smFISH_images/Eric_smFISH_images/20230608/DUSP1_Dex_10nM_75min_050223_R3' ,
        'smFISH_images/Eric_smFISH_images/20230608/DUSP1_Dex_10nM_90min_050223_R3' ,
        'smFISH_images/Eric_smFISH_images/20230608/DUSP1_Dex_10nM_120min_050223_R3' ,
        'smFISH_images/Eric_smFISH_images/20230608/DUSP1_Dex_10nM_180min_050223_R3' )
# Parameters
Condition = 'DUSP1_timesweep'      # Experimental condition. The options are:  'GR_timesweep' , 'DUSP1_timesweep'
Replica = 'L'                   # Replica name. 
time_list  = [0, 30, 50, 75, 90, 120, 180, 30, 50, 90, 120, 180, 30, 50, 75, 90, 120, 180]        # Time of image acquisition
DexConc_list = [0, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 1, 1, 1, 1, 1, 10, 10, 10, 10, 10, 10]        # Dex concentration

DUSP1_TCS_R3 = [
    {
        "condition": Condition,
        "replica": Replica,
        "time": time,
        "Dex_Conc": DexConc
    }
    for time, DexConc in zip(time_list, DexConc_list)
]
# Output the result
print(DUSP1_TCS_R3)

In [ ]:
runner(initial_data_location, DUSP1_TCS_R3)


In [ ]:
# List of directories to process
initial_data_location=(
        'smFISH_images/Eric_smFISH_images/20220725/DUSP1_Dex_100nM_6hr_0min_072022' ,
        'smFISH_images/Eric_smFISH_images/20220725/DUSP1_Dex_100nM_6hr_150min_072022' ,
        'smFISH_images/Eric_smFISH_images/20220725/DUSP1_Dex_100nM_6hr_180min_072022' )
# Parameters
Condition = 'DUSP1_timesweep'      # Experimental condition. The options are:  'GR_timesweep' , 'DUSP1_timesweep' , 'DUSP1_TPL'
Replica = 'M'                   # Replica name. 
time_list  = [0, 150, 180]        # Time of image acquisition
DexConc_list = [0, 100, 100]        # Dex concentration

DUSP1_TS_R4 = [
    {
        "condition": Condition,
        "replica": Replica,
        "time": time,
        "Dex_Conc": DexConc
    }
    for time, DexConc in zip(time_list, DexConc_list)
]
# Output the result
print(DUSP1_TS_R4)

In [ ]:
runner(initial_data_location, DUSP1_TS_R4)


In [ ]:
# List of directories to process
initial_data_location=(
        'smFISH_images/Eric_smFISH_images/20220927/DUSP1_100nM_Dex_0min_081822' ,
        'smFISH_images/Eric_smFISH_images/20220927/DUSP1_100nM_Dex_30min_081822' ,
        'smFISH_images/Eric_smFISH_images/20220927/DUSP1_100nM_Dex_60min_081822' ,
        'smFISH_images/Eric_smFISH_images/20220928/DUSP1_100nM_Dex_90min_081822' ,
        'smFISH_images/Eric_smFISH_images/20220928/DUSP1_100nM_Dex_120min_081822' ,
        'smFISH_images/Eric_smFISH_images/20220928/DUSP1_100nM_Dex_150min_081822' ,
        'smFISH_images/Eric_smFISH_images/20220929/DUSP1_100nM_Dex_180min_081822' )
# Parameters
Condition = 'DUSP1_timesweep'      # Experimental condition. The options are:  'GR_timesweep' , 'DUSP1_timesweep'
Replica = 'N'                   # Replica name. 
time_list  = [0, 30, 60, 90, 120, 150, 180]        # Time of image acquisition
DexConc_list = [0, 100, 100, 100, 100, 100, 100]        # Dex concentration

DUSP1_TS_R5 = [
    {
        "condition": Condition,
        "replica": Replica,
        "time": time,
        "Dex_Conc": DexConc
    }
    for time, DexConc in zip(time_list, DexConc_list)
]
# Output the result
print(DUSP1_TS_R5)

In [ ]:
runner(initial_data_location, DUSP1_TS_R5)


## ```DUSP1_TPL```  experiments

In [ ]:
# List of directories to process
initial_data_location=(
        'smFISH_images/Eric_smFISH_images/20221019/DUSP1_Dex_TPL_3hr_0min_Control_101422' ,
        'smFISH_images/Eric_smFISH_images/20221019/DUSP1_Dex100nM_75min_Control_101422' ,
        'smFISH_images/Eric_smFISH_images/20221019/DUSP1_Dex100nM_75min_TPL5uM_10min_101422' ,
        'smFISH_images/Eric_smFISH_images/20221019/DUSP1_Dex100nM_75min_TPL5uM_30min_101422' ,
        'smFISH_images/Eric_smFISH_images/20221019/DUSP1_Dex100nM_75min_TPL5uM_60min_101422' ,
        'smFISH_images/Eric_smFISH_images/20221020/DUSP1_Dex100nM_150min_Control_101422' ,
        'smFISH_images/Eric_smFISH_images/20221020/DUSP1_Dex100nM_150min_TPL5uM_10min_101422' ,
        'smFISH_images/Eric_smFISH_images/20221020/DUSP1_Dex100nM_150min_TPL5uM_30min_101422' ,
        'smFISH_images/Eric_smFISH_images/20221020/DUSP1_Dex100nM_150min_TPL5uM_60min_101422' )
# Parameters
Condition = 'DUSP1_TPL'      # Experimental condition. The options are:  'GR_timesweep' , 'DUSP1_timesweep' , 'DUSP1_TPL'
Replica = 'O'                   # Replica name. 
time_list  = [0, 75, 85, 105, 135, 150, 160, 180, 210]        # Time of image acquisition
DexConc_list = [0, 100, 100, 100, 100, 100, 100, 100, 100]        # Dex concentration
time_TPL = [None, None, 75, 75, 75, None, 150, 150, 150]  # Time of TPL addition relitive to Dex timepoint

DUSP1_TPL_R1 = [
    {
        "condition": Condition,
        "replica": Replica,
        "time": time,
        "Dex_Conc": DexConc,
        "time_tpl": tpl_time
    }
    for time, DexConc, tpl_time in zip(time_list, DexConc_list, time_TPL)
]
# Output the result
print(DUSP1_TPL_R1)

In [ ]:
runner(initial_data_location, DUSP1_TPL_R1)


In [ ]:
# List of directories to process
initial_data_location=(
        'smFISH_images/Eric_smFISH_images/20221103/DUSP1_Dex_0min_TPL_0min_110222' ,
        'smFISH_images/Eric_smFISH_images/20221103/DUSP1_Dex_0min_TPL_15min_110222' ,
        'smFISH_images/Eric_smFISH_images/20221103/DUSP1_Dex_0min_TPL_30min_110222' ,
        'smFISH_images/Eric_smFISH_images/20221103/DUSP1_Dex_0min_TPL_60min_110222' ,
        'smFISH_images/Eric_smFISH_images/20221107/DUSP1_Dex_20min_TPL_0min_110222' ,
        'smFISH_images/Eric_smFISH_images/20221107/DUSP1_Dex_20min_TPL_15min_110222' ,
        'smFISH_images/Eric_smFISH_images/20221107/DUSP1_Dex_20min_TPL_30min_110222' ,
        'smFISH_images/Eric_smFISH_images/20221107/DUSP1_Dex_20min_TPL_60min_110222' ,
        'smFISH_images/Eric_smFISH_images/20221107/DUSP1_Dex_75min_TPL_0min_110222' ,
        'smFISH_images/Eric_smFISH_images/20221107/DUSP1_Dex_75min_TPL_15min_110222' ,
        'smFISH_images/Eric_smFISH_images/20221107/DUSP1_Dex_75min_TPL_30min_110222' ,
        'smFISH_images/Eric_smFISH_images/20221107/DUSP1_Dex_75min_TPL_60min_110222' ,
        'smFISH_images/Eric_smFISH_images/20221109/DUSP1_Dex_180min_TPL_0min_110222' ,
        'smFISH_images/Eric_smFISH_images/20221129/DUSP1_Dex_180min_TPL_15min_110222' ,
        'smFISH_images/Eric_smFISH_images/20221129/DUSP1_Dex_180min_TPL_30min_110222' ,
        'smFISH_images/Eric_smFISH_images/20221129/DUSP1_Dex_180min_TPL_60min_110222' )
# Parameters
Condition = 'DUSP1_TPL'      # Experimental condition. The options are:  'GR_timesweep' , 'DUSP1_timesweep' , 'DUSP1_TPL'
Replica = 'P'                   # Replica name. 
time_list  = [0, 15, 30, 60, 20, 35, 50, 80, 75, 90, 105, 135, 180, 195, 210, 240]        # Time of image acquisition
DexConc_list = [0, 0, 0, 0, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100]        # Dex concentration
time_TPL = [None, 0, 0, 0, None, 20, 20, 20, None, 75, 75, 75, None, 180, 180, 180]  # Time of TPL addition relitive to Dex timepoint

DUSP1_TPL_R2 = [
    {
        "condition": Condition,
        "replica": Replica,
        "time": time,
        "Dex_Conc": DexConc,
        "time_tpl": tpl_time
    }
    for time, DexConc, tpl_time in zip(time_list, DexConc_list, time_TPL)
]
# Output the result
print(DUSP1_TPL_R2)

In [ ]:
runner(initial_data_location, DUSP1_TPL_R2)
